In [1]:
from flask import Flask, request
from flask_restful import Resource, Api
import pymysql
import pandas as pd

In [2]:
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='',
    db='tutorcatdb'
)

# Define the SQL query to retrieve data from a table
query = 'SELECT * FROM ai_table;'

# Use pandas to load the query results into a DataFrame
ai_df = pd.read_sql(query, connection)

connection.close()

# Print the DataFrame
print(ai_df.head())

   NotesID  NotesID2       Val
0       25        25  1.000000
1       26        25  0.650945
2       27        25 -0.615462
3       28        25 -0.977140
4       29        25  0.225494


In [3]:
piv_dataFrame = ai_df.pivot_table(index='NotesID', columns='NotesID2', values="Val").fillna(0)

piv_dataFrame

NotesID2,25,26,27,28,29,30,31
NotesID,,,,,,,
25,1.000000,0.650945,-0.615462,-0.977140,0.225494,0.225494,0.225494
26,0.650945,1.000000,-0.457496,-0.577350,0.577350,0.577350,-0.577350
27,-0.615462,-0.457496,1.000000,0.440225,0.440225,0.440225,0.088045
28,-0.977140,-0.577350,0.440225,1.000000,-0.333333,-0.333333,-0.333333
29,0.225494,0.577350,0.440225,-0.333333,1.000000,1.000000,-0.333333
30,0.225494,0.577350,0.440225,-0.333333,1.000000,1.000000,-0.333333
31,0.225494,-0.577350,0.088045,-0.333333,-0.333333,-0.333333,1.000000


In [4]:
def get_similarity_notes(notesID, user_rating):
    similarity_mark = piv_dataFrame[notesID] * user_rating
    similarity_mark = similarity_mark.sort_values(ascending=False)

    return similarity_mark

print(get_similarity_notes(25, 5))

NotesID
25    5.000000
26    3.254725
29    1.127470
30    1.127470
31    1.127470
27   -3.077310
28   -4.885700
Name: 25, dtype: float64


In [5]:
notes_lover = [(25, 1), (26, 1), (27, 1)]

# piv_dataFrame_null = pd.DataFrame()
#
# for notesID,rating in notes_lover:
#     piv_dataFrame_null = piv_dataFrame.append(get_similarity_notes(notesID,rating),ignore_index=True)
#
# piv_dataFrame_null.head()
#
# piv_dataFrame_null.sum().sort_values(ascending = False)

In [ ]:
from flask import Flask, jsonify

app = Flask(__name__)


@app.route('/recommend/<int:student_id>', methods=['GET'])
def recommend(student_id):
    # req_data = request.get_json()
    # student_id = req_data['student_id']

    connection = pymysql.connect(
        host='localhost',
        user='root',
        password='',
        db='tutorcatdb'
    )

    query = "SELECT NotesID,Rating FROM notes_rating WHERE StudentID={} ORDER BY Rating;".format(student_id)

    student_rating_df = pd.read_sql(query, connection)

    # Close the database connection
    connection.close()

    notes_id_list = [(row['NotesID'], 5 - row['Rating']) for _, row in student_rating_df.iterrows()]
    print(notes_id_list)
    print(type(notes_id_list))

    piv_Series_null = pd.DataFrame()

    for notesID,rating in notes_lover:
        piv_Series_null = piv_dataFrame.append(get_similarity_notes(notesID, rating), ignore_index=True)

    piv_Series_null.head()

    piv_Series_null= piv_Series_null.sum().sort_values(ascending = False)
    print(type(piv_Series_null))
    print(piv_Series_null)
    piv_dataFrame_null = piv_Series_null.to_frame(name='Val').rename_axis('NotesID')

    print(type(piv_dataFrame_null))
    print(piv_dataFrame_null)

    onlyIDlist = set([x[0] for x in notes_id_list])
    print(onlyIDlist)


    piv_dataFrame_null = piv_dataFrame_null[~piv_dataFrame_null.index.isin(onlyIDlist)]

    print(piv_dataFrame_null)

    notes_ids_list = piv_dataFrame_null.index.tolist()
    json_str = jsonify(notes_ids_list)
    # recommendations = ['Product 1', 'Product 2', 'Product 3']
    # return {'recommendations': recommendations}

    return json_str

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [16/Apr/2023 05:37:49] "GET /recommend/10003 HTTP/1.1" 200 -


[(25, 1), (26, 1), (27, 1), (29, 1), (30, 1)]
<class 'list'>
<class 'pandas.core.series.Series'>
29    3.016628
30    3.016628
27    2.335762
26    0.735953
25    0.119363
31   -0.175765
28   -0.674039
dtype: float64
<class 'pandas.core.frame.DataFrame'>
              Val
NotesID          
29       3.016628
30       3.016628
27       2.335762
26       0.735953
25       0.119363
31      -0.175765
28      -0.674039
{25, 26, 27, 29, 30}
              Val
NotesID          
31      -0.175765
28      -0.674039


127.0.0.1 - - [16/Apr/2023 05:37:58] "GET /recommend/10003 HTTP/1.0" 200 -


[(25, 1), (26, 1), (27, 1), (29, 1), (30, 1)]
<class 'list'>
<class 'pandas.core.series.Series'>
29    3.016628
30    3.016628
27    2.335762
26    0.735953
25    0.119363
31   -0.175765
28   -0.674039
dtype: float64
<class 'pandas.core.frame.DataFrame'>
              Val
NotesID          
29       3.016628
30       3.016628
27       2.335762
26       0.735953
25       0.119363
31      -0.175765
28      -0.674039
{25, 26, 27, 29, 30}
              Val
NotesID          
31      -0.175765
28      -0.674039


127.0.0.1 - - [16/Apr/2023 05:38:11] "GET /recommend/10003 HTTP/1.0" 200 -


[(25, 1), (26, 1), (27, 1), (29, 1), (30, 1)]
<class 'list'>
<class 'pandas.core.series.Series'>
29    3.016628
30    3.016628
27    2.335762
26    0.735953
25    0.119363
31   -0.175765
28   -0.674039
dtype: float64
<class 'pandas.core.frame.DataFrame'>
              Val
NotesID          
29       3.016628
30       3.016628
27       2.335762
26       0.735953
25       0.119363
31      -0.175765
28      -0.674039
{25, 26, 27, 29, 30}
              Val
NotesID          
31      -0.175765
28      -0.674039
